<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Regresión polinomial

Ejemplo de regresión polinomial<br>
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('FuelConsumptionCo2.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv > FuelConsumptionCo2.csv
    else:
        !wget FuelConsumptionCo2.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv

### `FuelConsumption.csv`:
El dataset **`FuelConsumption.csv`** contiene el consumo específico por tipo de vehículo y la emisión estimado de dioxido de carbono (Co2) de nuevos vehículos que son venidos en canada.<br> [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** --> ejemplo 2014
- **MAKE** --> ejemplo Acura
- **MODEL** --> ejemplo ILX
- **VEHICLE CLASS** --> ejemplo SUV
- **ENGINE SIZE** --> ejemplo 4.7
- **CYLINDERS** --> ejemplo 6
- **TRANSMISSION** --> ejemplo A6
- **FUEL CONSUMPTION in CITY(L/100 km)** --> ejemplo 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** --> ejemplo 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** --> ejemplo 9.2
- **CO2 EMISSIONS (g/km)** --> ejemplo 182

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("FuelConsumptionCo2.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

## Fin de la limpieza
Se finalizó la limpieza, no hay datos mal cargados o incompletos en este dataset

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Analizaremos como se compartan algunos atributos de entrada contra el objetivo (las emisiones, el Co2)
pp = sns.pairplot(data=df, x_vars=['CYLINDERS', 'ENGINESIZE', 'FUELCONSUMPTION_COMB'], y_vars=['CO2EMISSIONS'], diag_kind=None, height=5)
plt.show()


Se puede observar que todas tienen una relación lineal con le emisión de CO2, pero para este ejemplo nos quedaremos con las cilindradas del motor para evaluar contra el CO2. Queda en la voluntad del alumno de seguir experimentando con otros features

In [ ]:
# Analizamos la distribución de la variable que se desea analizar
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.boxplot(x=df['CYLINDERS'], ax=ax)
ax.grid('dashed')

Se puede observar que está centrada a la izquierda la distribución y no hay outliers, por lo que no habrá problema. El hecho de que la distribución esté marcada a la izquierda querra decir que hay menos datos para analizar de alta cilindrada.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df['CYLINDERS'], y=df['CO2EMISSIONS'], color='darkBlue', ax=ax)
ax.grid('dashed')
plt.show()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
X = df[['CYLINDERS']].values
y = df['CO2EMISSIONS'].values

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Crear un modelo de regresión polinominal

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

mse_train_list = []
mse_test_list = []
degrees_to_plot = [1, 3, 10]
max_dregree = max(degrees_to_plot)

plt.figure(figsize=(14, 5))
for degree in range(1, (max_dregree+1)):

    plot_number = 0
    try:
        plot_number = degrees_to_plot.index(degree) + 1
    except:
        plot_number = 0

    if plot_number > 0:
        ax = plt.subplot(1, len(degrees_to_plot), plot_number)
        plt.setp(ax, xticks=(), yticks=())

    polynomial_features = PolynomialFeatures()
    poly = PolynomialFeatures(degree=degree)
    
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.fit_transform(X_test)

    lr = LinearRegression()
    lr.fit(X_train_poly, y_train)
    y_hat = lr.predict(X_test_poly)
    y_hat_train = lr.predict(X_train_poly)

    mse = mean_squared_error(y_test, y_hat)
    mse_train = mean_squared_error(y_train, y_hat_train)

    mse_train_list.append(mse_train)
    mse_test_list.append(mse)

    if plot_number > 0:

        lx = sorted(X_test)
        lx_poly = poly.fit_transform(lx)
        ly = lr.predict(lx_poly)

        plt.plot(lx, ly, label="Modelo", color='darkGreen')
        plt.scatter(X_train, y_train, edgecolor='b', s=20, label="data")
        plt.xlabel("cilindrada")
        plt.ylabel("emisiones")
        plt.legend()
        plt.title("Grado {}\nMSE = {:.3f}".format(degree, mse))

plt.show()

In [ ]:
plt.plot(range(1, (max_dregree+1)), mse_train_list, c='g', label="train")
plt.plot(range(1, (max_dregree+1)), mse_test_list, c='b', label="test")
plt.xlabel("modelo")
plt.ylabel("Error")
plt.legend()
plt.title("Nivel óptimo de complejidad del modelo")
plt.show()
print('Nivel óptimo:', mse_test_list.index(min(mse_test_list))+1)

Construir el modelo que mejor ajuste al problema

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

polynomial_features = PolynomialFeatures()
poly = PolynomialFeatures(degree=3)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

lr = LinearRegression()
lr.fit(X_train_poly, y_train)
y_hat = lr.predict(X_test_poly)

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
from sklearn.metrics import r2_score
score = r2_score(y_test, y_hat)
print(f"Coeficiente de determinación: {score:.2f}")

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
lx = sorted(X_test)
lx_poly = poly.fit_transform(lx)
ly = lr.predict(lx_poly)

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot()

ax.plot(lx, ly, label="polinomial", color='darkGreen')
ax.scatter(X_train, y_train, edgecolor='b', s=20, label="data")
ax.set_xlabel("cilindrada")
ax.set_ylabel("emisiones")
ax.legend()
ax.grid('dashed')
plt.show()

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

En este ejemplo emisiones de CO2 se puede realizar todos los tipos de regresioens estudiadas hasta el momento, en este caso solo se comparó contra la cilindrada utilizando una gresión polinomial

#### Exportar

In [ ]:
import pickle
pickle.dump(lr, open('alquileres.pkl','wb'))